In [1]:
import numpy as np
import os
import re

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = 'cuda'

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
import ast

In [4]:
dtype = torch.float16
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_compute_dtype=dtype,
)

In [5]:
# model_id = "codellama/CodeLlama-7b-Python-hf"
model_id = "codellama/CodeLlama-7b-Instruct-hf"

llm_tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto",       
    quantization_config=bnb_config,         
).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
context = ''' 
def f(a, b):
    if a > b:
        return a
    else:
        return b
'''

context = '''
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import difflib

# Sample DataFrame setup
data = {
    'id': [1, 2, 3],
    'text': ['This is a sample text.', 'This is another example text.', 'Here is a completely different text.']
}
df = pd.DataFrame(data)

# Function to display the selected texts and highlight differences
def display_texts_and_diff(id1, id2):
    text1 = df.loc[df['id'] == id1, 'text'].values[0]
    text2 = df.loc[df['id'] == id2, 'text'].values[0]
    
    # Compute the differences
    d = difflib.Differ()
    diff = list(d.compare(text1.split(), text2.split()))
    diff_html = '<table><tr><td>{}</td><td>{}</td></tr>'.format(
        ' '.join('<span style="background-color: #ffcccc">{}</span>'.format(word.strip('-+')) if word.startswith('-') else word for word in diff if not word.startswith('+')),
        ' '.join('<span style="background-color: #ccffcc">{}</span>'.format(word.strip('-+')) if word.startswith('+') else word for word in diff if not word.startswith('-'))
    )
    
    display(HTML(diff_html))

# Create dropdowns for ID selection
id_dropdown1 = widgets.Dropdown(options=df['id'], description='ID 1:')
id_dropdown2 = widgets.Dropdown(options=df['id'], description='ID 2:')

# Button to trigger the display function
button = widgets.Button(description="Compare Texts")

# Display widgets
display(id_dropdown1, id_dropdown2, button)

# Button click event handler
def on_button_clicked(b):
    display_texts_and_diff(id_dropdown1.value, id_dropdown2.value)

button.on_click(on_button_clicked)
'''

question = 'Describe what the code is doing in one sentence'

prompt = f"""You are given the <CONTEXT/> of python code and a <QUESTION/>. ANSWER the QUESTION.
<CONTEXT>
{context}
</CONTEXT>
<QUESTION>
{question}
</QUESTION>
<ANSWER>
"""

In [7]:
model_input = llm_tokenizer(prompt, return_tensors="pt").to('cuda')
n_tokens = model_input['input_ids'].shape[1]

In [9]:
from tqdm import tqdm
# for _ in tqdm(range(1000)):
    # with torch.no_grad():
output = llm_model.generate(**model_input, max_new_tokens=300, pad_token_id=llm_tokenizer.eos_token_id)

In [10]:
output = output[0, n_tokens:]
output = llm_tokenizer.decode(output, skip_special_tokens=True)
output = output.split('</ANSWER>')[0].strip()

In [11]:
print(output)

The code is creating a dropdown menu for selecting two IDs from a dataframe, and then displaying the corresponding text values side-by-side with highlighted differences.
